In [1]:
import os
import zipfile
from sqlalchemy import create_engine

import pandas as pd

In [2]:
from tkgqa_generator.tkgqa_generator.constants import DATA_ICEWS_EVENTS_DATA_DIR

In [3]:
DATA_ICEWS_EVENTS_DATA_DIR

PosixPath('/home/pascal/PhD/TemporalLogicKGQA/tkgqa_generator/data/ICEWS/ICEWS Coded Event Data')

In [4]:
# loop the folder, and unzip all the files ending with .zip
# it will override the data if it is running twice
for file in os.listdir(DATA_ICEWS_EVENTS_DATA_DIR):
    if file.endswith(".zip"):
        # unzip the file
        zip_path = DATA_ICEWS_EVENTS_DATA_DIR / file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DATA_ICEWS_EVENTS_DATA_DIR)

In [5]:
# read all .tab or .csv files into the df and check their column distribution
# pandas read from tab, csv files
# have done the check, all files have the consistent same column names
combined_df = None
for file in os.listdir(DATA_ICEWS_EVENTS_DATA_DIR):
    if file.endswith(".tab") or file.endswith(".csv"):
        tab_path = DATA_ICEWS_EVENTS_DATA_DIR / file
        df = pd.read_csv(
            tab_path, sep="\t", low_memory=False
        )
        if combined_df is None:
            combined_df = df
        else:
            # combine df and combined_df
            combined_df = pd.concat(
                [combined_df, df], ignore_index=True
            )

In [6]:
# combined_df.to_csv(
#     "ICEWS.csv"
# )

In [7]:
combined_df

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,37756867,2023-03-15,Organized Crime (Brazil),"Criminals / Gangs,Dissident",Brazil,Make statement,10,0.0,Indigenous People (Brazil),"General Population / Civilian / Social,Social",Brazil,57081171,2,Unknown,NaN,NaN,Estado de Roraima,Brazil,1.0000,-61.0000
1,37756337,2023-03-17,Undersecretary (United States),Government,United States,Make statement,10,0.0,Iran,NaN,Iran,57079767,3,The Jerusalem Post,NaN,NaN,NaN,United States,38.8951,-77.0364
2,37756338,2023-03-17,International Atomic Energy Agency,"Global,Global Diplomatic IGOs,Global Energy IG...",NaN,Make statement,10,0.0,Iran,NaN,Iran,57079767,5,The Jerusalem Post,NaN,NaN,NaN,Iran,35.6944,51.4215
3,37756289,2023-03-17,Citizen (Israel),"General Population / Civilian / Social,Social",Israel,Demonstrate or rally,141,-6.5,Israel,NaN,Israel,57079770,1,The Jerusalem Post,NaN,NaN,NaN,Israel,31.7790,35.2253
4,37750848,2023-03-18,Aam Aadmi Party,"(National) Major Party,Far Left,Ideological,Pa...",India,Criticize or denounce,111,-2.0,Bharatiya Janata,"(National) Major Party,Opposition Major Party ...",India,57068117,1,Hindustan Times,Delhi,NaN,National Capital Territory of Delhi,India,28.6538,77.2290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18662553,37711733,2023-03-13,Macky Sall,"(National) Minor Party,Center Left,Elite,Execu...",Senegal,Make statement,10,0.0,Kaïs Saïed,"Executive,Executive Office,Government",Tunisia,56995482,1,All Africa,NaN,NaN,NaN,Tunisia,36.8190,10.1658
18662554,37711734,2023-03-13,World Bank,"Global,Global Development IGOs,Global Economic...",NaN,Reduce relations,160,-4.0,Tunisia,NaN,Tunisia,56995482,6,All Africa,NaN,NaN,NaN,Tunisia,36.8190,10.1658
18662555,37711688,2023-03-13,Abdoulaye Bathily,"(National) Minor Party,Far Left,Government,Ide...",Senegal,Make an appeal or request,20,3.0,Libya,NaN,Libya,56995483,1,All Africa,Tripoli,NaN,Sha`biyat Tarabulus,Libya,32.8752,13.1875
18662556,37711625,2023-03-13,Police (Tunisia),"Government,Police",Tunisia,"Arrest, detain, or charge with legal action",173,-5.0,Student (Tunisia),"Education,Social",Tunisia,56995487,4,All Africa,NaN,NaN,NaN,Tunisia,36.8190,10.1658


In [8]:
db_name = "tkgqa"
db_user = "tkgqa"
db_pass = "tkgqa"
db_host = "localhost"  # or the IP address of your database server
db_port = "5433"
connection_string = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"

In [9]:
engine = create_engine(connection_string)

In [10]:
combined_df.to_sql(
    "icews", engine, if_exists="append", index=False
    
)

558